In [1]:
import torch
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer
from transformers import BertPreTrainedModel, BertModel, DistilBertPreTrainedModel, DistilBertModel, DistilBertConfig
from transformers import RobertaTokenizer

import pandas as pd
import numpy as np
import os
import csv

from sklearn.metrics import classification_report

# just checking somethingdf
from datasets import Dataset

# my custom function
import utils
import importlib
from sklearn.model_selection import train_test_split

# import the libraries from bert
from bert import BERT_PCL, PCLDataset, Trainer_PCL, DistillBert_PCL, RobertaPCL
import bert

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  DEVICE = 'cpu'
else:
  DEVICE = 'cuda:0'

import os
import pickle
from itertools import product
content = os.path.abspath('/vol/bitbucket/ahf119/')

In [2]:
importlib.reload(bert)

<module 'bert' from '/homes/ahf119/Desktop/nlp_cw/bert.py'>

In [3]:
EPOCHS = 7

In [4]:
def write_dict_to_csv(file_path, data_dict):
    # Extract fieldnames from the first dictionary key
    fieldnames = list(data_dict.keys())

    # Check if the file already exists or not
    file_exists = True
    try:
        with open(file_path, 'r') as file:
            reader = csv.DictReader(file)
            if not reader.fieldnames:
                file_exists = False
    except FileNotFoundError:
        file_exists = False

    # Write data to CSV
    with open(file_path, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Write header if the file is empty
        if not file_exists:
            writer.writeheader()

        # Write data
        writer.writerow(data_dict)

In [5]:
params = {
    'lr': [0.00001, 0.00005],
    # 'warmup_ratio': [0.1, 0.2],
    'learning_schedule': ['linear', 'constant'],
    'augmentation': {
        'augment0': {
            'inserted': False,
            'subbed': False,
            'back_translated': False,
            'deleted': False,
            'swapped': False
        },
        'augment1': {        
            'inserted': True,
            'subbed': False,
            'back_translated': False,
            'deleted': False,
            'swapped': False
            },
            'augment2': {
                'inserted': False,
                'subbed': True,
                'back_translated': False,
                'deleted': False,
                'swapped': False
            },
            'augment3': {
                'inserted': False,
                'subbed': False,
                'back_translated': True,
                'deleted': False,
                'swapped': False
            },
            'augment4': {
                'inserted': False,
                'subbed': False,
                'back_translated': False,
                'deleted': True,
                'swapped': False
            },
            'augment5': {
                'inserted': False,
                'subbed': False,
                'back_translated': False,
                'deleted': False,
                'swapped': True
            }
        }
    }

# Generate all possible combinations of parameters
param_grid = list(product(*params.values()))
print(len(param_grid))

24


In [6]:
def train_once(lr, learning_schedule, augment_dict, best_f1):
    model_output=f'{content}/wandtest_model'
    model_type = 'bert-base-cased'
    
    if model_type.startswith('bert'):
        model = BERT_PCL.from_pretrained(model_type)
    elif model_type.startswith('distil'):
        model = DistillBert_PCL.from_pretrained(model_type)
    elif model_type.startswith('roberta'):
        model = RobertaPCL.from_pretrained(model_type)
    
    args = TrainingArguments(
        output_dir=f'{content}/wandtest',
        learning_rate=lr,
        logging_steps=10,
        remove_unused_columns=False,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=EPOCHS,
        seed=42,
        evaluation_strategy='steps',
        eval_steps=100,
        save_strategy='steps',
        save_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        greater_is_better=True,
        lr_scheduler_type=learning_schedule,
        warmup_ratio=0.1,
    )

    train_dataset, val_dataset, test_dataset = bert.all_augmentations(model_type, inserted = augment_dict['inserted'], subbed = augment_dict['subbed'], back_translated = augment_dict['back_translated'], 
                                                                      deleted = augment_dict['deleted'], swapped = augment_dict['swapped'])

    trainer = bert.main_PCL_v1(model=model, args=args, train_dataset=train_dataset, val_dataset=val_dataset, model_output=model_output)
    if model_type.startswith('bert'):
        model = BERT_PCL.from_pretrained(model_output)
    elif model_type.startswith('distil'):
        model = DistillBert_PCL.from_pretrained(model_output)
    elif model_type.startswith('roberta'):
        model = RobertaPCL.from_pretrained(model_output)
    metrics = bert.evaluate(model, DataLoader(val_dataset))
    f1 = metrics['PCL']['f1-score']

    if f1 > best_f1:
        best_f1 = f1
        trainer.save_model(f'{content}/best_model')
        with open('saved_metrics.pkl', 'wb') as f:
            pickle.dump(metrics, f)

    model_data = metrics['PCL'].copy()
    model_data['learning_rate'] = lr
    model_data['learning_schedule'] = learning_schedule
    model_data['inserted'] = augment_dict['inserted']
    model_data['deleted'] = augment_dict['deleted']
    model_data['subbed'] = augment_dict['subbed']
    model_data['back_translated'] = augment_dict['back_translated']
    model_data['swapped'] = augment_dict['swapped']

    write_dict_to_csv(file_path = 'hyperparam_res_{}.csv'.format(model_type), data_dict = model_data)
    
    return best_f1
    

In [7]:
best_f1 = -1

current_parameter = 0
for i, param_set in enumerate(param_grid):
    if i < current_parameter:
        continue
    best_f1 = train_once(param_set[0], param_set[1], params['augmentation'][param_set[2]], best_f1)
    print(f'Finished param set with index: {i}')

Some weights of BERT_PCL were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classification.1.bias', 'classification.1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


inserted False
subbed False
back_translated False
deleted False
swapped False


wandb: Currently logged in as: anders-frelin19 (hugo-frelin). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.391300,0.452175,0.897732,0.000000,0.000000,0.000000
200,0.339600,0.380004,0.903303,0.554688,0.276265,0.368831
300,0.359800,0.410615,0.906088,0.629630,0.198444,0.301775
400,0.318800,0.318992,0.902507,0.523256,0.525292,0.524272
500,0.204500,0.389605,0.916832,0.653846,0.396887,0.493947
600,0.262500,0.383187,0.909670,0.566372,0.498054,0.530021
700,0.174400,0.395918,0.912455,0.592040,0.463035,0.519651
800,0.170700,0.384158,0.912455,0.582960,0.505837,0.541667
900,0.090900,0.470213,0.913649,0.603093,0.455253,0.518847
1000,0.103200,0.466960,0.911659,0.578475,0.501946,0.537500


/vol/bitbucket/ahf119/nlp_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

inserted False
subbed False
back_translated False
deleted False
swapped True


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.399100,0.347497,0.909272,0.635514,0.264591,0.373626
200,0.359600,0.390731,0.789097,0.298969,0.789883,0.433761
300,0.178300,0.311135,0.894150,0.484642,0.552529,0.516364
400,0.171500,0.395407,0.907282,0.562500,0.420233,0.481069
500,0.120200,0.432937,0.900915,0.515504,0.517510,0.516505
600,0.107200,0.496840,0.879029,0.427692,0.540856,0.477663
700,0.052700,0.800219,0.911261,0.634921,0.311284,0.417755
800,0.047300,0.707041,0.912853,0.620253,0.381323,0.472289


Checkpoint destination directory /vol/bitbucket/ahf119/wandtest/checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory /vol/bitbucket/ahf119/wandtest/checkpoint-200 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory /vol/bitbucket/ahf119/wandtest/checkpoint-300 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory /vol/bitbucket/ahf119/wandtest/checkpoint-400 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory /vol/bitbucket/ahf119/wandtest/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory /vol/bitbucket/ahf119/wandtest/checkpoint-600 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Chec

  0%|          | 0/2513 [00:00<?, ?it/s]

Finished param set with index: 23
